# Schema.org GraphQL

## Proposal

This document proposes some directions and criteria that will guide the implementation of *Schema.org* using *GraphQL*. 

Objectively, the general proposal is (could be replicated to _graphql-schema-org_ lib):

* use `__typename` value from database and for input to be used for `resolve` method.
* convert primitive Schema.org data types to GraphQL primitive/scalar types.
* when a field is defined by more than one primitive/scalar type (and just by primitive/scalar types), it will be converted to `GraphQLString` type.
* when a field is defined by one or more primitive/scalar type and object types, descard alll primitive/scalar types and keep just the object types.
* when there are more than one object type, create an `union type` using the field's name and specify all types into that. Assign the `union type` created to the respective field.
* apply GraphQLList for each field with the respective type (e.g. `GraphQLList(author)`), except for `identifier` field (it should be just a `GraphQLString`).
* As GraphQL doesn't support union for input types, use _graphql-union-input-type_ (see: https://github.com/Cardinal90/graphql-union-input-type/issues/9)

Internally, the implementation could:

* use `identifier` composed by `context` + `typename` + `sequential number`. That also will help `resolve` method when there is no `__typename` information.


## Data Extraction

It is possible to get schema.org data structure on [doc developers' web page](https://schema.org/docs/developers.html).

The file formats available are:

* Triples
* Quads
* JSON-LD
* RDF/XML
* Turtle
* CSV

_To see an example of each file, check it out at the end of this document._

### Implementation

Another way to get data from schema.org is using a `web scrapping` technique. 
Maybe it is not the more robust way to do that (regarding of all these files availables), 
but just for the creating of GraphQL types with their fields, `web scrapping` would be easier.
When the current proposal is approved, the `extraction` method used here could be changed 
and could use one of the other file format presented before.  

The approach used here will:

* get data from `https://schema.org/<TypeName>`
* parsing for data inside a table
* check if the data corresponding to the desired table
* convert primitive Schema.org data types to GraphQL data tyeps
* get other types dependents
* create `GraphQL Definition/Output` types
* create `GraphQL Union` types
* create `GraphQL Input` type

In [3]:
# https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
import os
import itertools
import traceback as tb
from jinja2 import Template
import lxml.html as lh
import pandas as pd
import requests

In [4]:
def get_all_types():
    url = 'https://schema.org/version/latest/all-layers-types.csv'
    df = pd.read_csv(url)
    return df['label']

typenames = get_all_types()

In [5]:
def extract_schema(url, dir_path='/home/xmn/dev/tmp/schemas/content'):
    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, url.split('/')[-1])
    
    if not os.path.isfile(file_path):
        # Create a handle, page, to handle the contents of the website
        page = requests.get(url)
        content = page.content
    
        with open(file_path, 'wb') as f:
            f.write(content)
    else:
        with open(file_path, 'rb') as f:
            content = f.read()

    # Store the contents of the website under doc
    doc = lh.fromstring(content)
    
    # Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//table[@class="definition-table"]//tr')
    
    if not isinstance(tr_elements, list) or not len(tr_elements):
        # Property; Expected Type; Description
        # TODO: check if this approach is good
        print(('[EE] Something wrong with the page {}.'.format(url)))
        col = [
            ('Property', 'value'), 
            ('Expected Type', 'Text'), 
            ('Description', 'TODO: check better how to deal if this field/type')
        ]
        return pd.DataFrame({title: column for (title, column) in col})
        
    
    # Create empty list
    col = []
    i = 0
    # For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i += 1
        name = t.text_content()
        col.append((name,[]))
    
    #Since out first row is the header, data is stored on the second row onwards
    run = True
    for j in range(1,len(tr_elements)):
        # T is our j'th row
        T = tr_elements[j]

        # If row is not of size 3, the //tr data is not from our table 
        if len(T) != 3:
            continue

        # check end of the table
        for t in T.iterchildren():
            data = t.text_content()
            
            if 'On Types' in str(data):
                run = False
                break
        
        if not run:
            break
            
        # i is the index of our column
        i = 0
        
        # Iterate through each element of the row
        for t in T.iterchildren():
            data = t.text_content() 
            # Check if row is empty
            if i > 0:
            # Convert any numerical value to integers
                try:
                    data = int(data)
                except:
                    ...
            # Append the data to the empty list of the i'th column
            col[i][1].append(data)
            # Increment i for the next column
            i += 1
            
    return pd.DataFrame({title: column for (title, column) in col})

In [6]:
def get_schema_url(typename):
    return 'https://schema.org/{}'.format(typename)

In [7]:
def get_schema_df(typename):
    df = extract_schema(get_schema_url(typename))
    df = df.applymap(lambda v: v.strip())
    # using a non-public API
    df._metadata = {'typename': typename}
    return df
    

df_ori = get_schema_df('Dataset')
df_ori.head(10)

,Property,Expected Type,Description
0,distribution,DataDownload,"A downloadable form of this dataset, at a spec..."
1,includedInDataCatalog,DataCatalog,A data catalog which contains this dataset. Su...
2,issn,Text,The International Standard Serial Number (ISSN...
3,measurementTechnique,Text or URL,A technique or technology used in a Dataset (o...
4,variableMeasured,PropertyValue or Text,The variableMeasured property can indicate (re...
5,about,Thing,The subject matter of the content. Inverse pro...
6,accessMode,Text,The human sensory perceptual system or cogniti...
7,accessModeSufficient,Text,A list of single or combined accessModes that ...
8,accessibilityAPI,Text,Indicates that the resource is compatible with...
9,accessibilityControl,Text,Identifies input methods that are sufficient t...


In [8]:
# Schema Data Types -> GraphQL Types
schema_graphql_map = {
    'Text': 'GraphQLString',
    'URL': 'GraphQLString',
    'PropertyValue': 'GraphQLString',
    'Date': 'GraphQLString',
    'Number': 'GraphQLFloat',
    'Float': 'GraphQLFloat',
    'Integer': 'GraphQLInt',
    'Time': 'GraphQLString',
    'DateTime': 'GraphQLString',
    'Boolean': 'GraphQLBoolean',
    'CssSelectorType': 'GraphQLString',
    'XPathType': 'GraphQLString',
}

# GraphQL Types
graphql_primitive_types = (
    'GraphQLString',
    'GraphQLInt',
    'GraphQLFloat',
    'GraphQLBoolean',
)


def fix_expected_types(types):
    """fix_expected_types"""
    result = {
        schema_graphql_map[t] if t in schema_graphql_map
        else t
        for t in types
    }
    
    n_total = len(result)
    n_primitive = len([
        None for t in result if t in graphql_primitive_types
    ])
    
    if n_total > 1:
        if n_total == n_primitive:
            result = {'GraphQLString'}
        else:
            result = {
                t for t in result 
                if t not in graphql_primitive_types
            }
    return result

In [9]:
def schemaorg2graphql(df):
    # using a non-public API
    _metadata = df._metadata
    df = df.copy()
    df._metadata = _metadata
    
    # fix expected type
    df['Expected Type'] = df['Expected Type'].map(
        lambda v: fix_expected_types({t.strip() for t in v.split(' or ')})
    )
    return df

df = schemaorg2graphql(df_ori)
df.head(20)

,Property,Expected Type,Description
0,distribution,{DataDownload},"A downloadable form of this dataset, at a spec..."
1,includedInDataCatalog,{DataCatalog},A data catalog which contains this dataset. Su...
2,issn,{GraphQLString},The International Standard Serial Number (ISSN...
3,measurementTechnique,{GraphQLString},A technique or technology used in a Dataset (o...
4,variableMeasured,{GraphQLString},The variableMeasured property can indicate (re...
5,about,{Thing},The subject matter of the content. Inverse pro...
6,accessMode,{GraphQLString},The human sensory perceptual system or cogniti...
7,accessModeSufficient,{GraphQLString},A list of single or combined accessModes that ...
8,accessibilityAPI,{GraphQLString},Indicates that the resource is compatible with...
9,accessibilityControl,{GraphQLString},Identifies input methods that are sufficient t...


In [10]:
def get_graphql_schema_df(typename):
    """get_prepared_schema_df"""
    df = get_schema_df(typename)
    df = schemaorg2graphql(df)
    return df


def get_dependencies(df):
    """get_depencencies"""
    deps = df['Expected Type'].values.flatten()
    deps = set(itertools.chain.from_iterable(deps))
    deps = [d for d in deps if not d.startswith('GraphQL')]
    return set(deps)

In [27]:
# !rm /home/xmn/dev/tmp/schemas/content/*
# path = '/home/xmn/dev/tmp/schemas/content'

In [12]:
schemas = {df._metadata['typename']: df}

deps = get_dependencies(df)
# deps = list(typenames)
max_num_errors = 10
num_errors = 1

while len(deps):
    dep = deps.pop()
    if dep not in schemas:
        try:
            schemas[dep] = get_graphql_schema_df(dep)
            new_deps = get_dependencies(schemas[dep])
            deps = (deps | new_deps) - set(schemas.keys())
        except:
            print('[EE] {}: {}'.format(dep, tb.print_exc()))

In [13]:
len(schemas.keys()), schemas.keys()

(114,
 dict_keys(['Dataset', 'Event', 'Product', 'OfferItemCondition', 'Enumeration', 'Person', 'OfferCatalog', 'Organization', 'Language', 'PostalAddress', 'GeoShape', 'Comment', 'Question', 'ItemList', 'ImageObject', 'MediaSubscription', 'Demand', 'TypeAndQuantityNode', 'Property', 'VideoObject', 'DeliveryMethod', 'Offer', 'Review', 'AlignmentObject', 'ProgramMembership', 'CorrectionComment', 'GenderType', 'Occupation', 'QuantitativeValue', 'StructuredValue', 'Place', 'Map', 'GeospatialGeometry', 'EducationalOrganization', 'Class', 'PhysicalActivityCategory', 'PriceSpecification', 'NewsArticle', 'SpeakableSpecification', 'MonetaryAmount', 'QualitativeValue', 'MonetaryAmountDistribution', 'InteractionCounter', 'WebSite', 'SoftwareApplication', 'DataFeed', 'Rating', 'DataDownload', 'ProductModel', 'DataCatalog', 'EventStatusType', 'AudioObject', 'BusinessEntityType', 'ContactPoint', 'AdministrativeArea', 'OpeningHoursSpecification', 'DayOfWeek', 'MediaObject', 'MusicGroup', 'MusicAlbum

In [14]:
def get_union_types(df):
    df = df.copy()
    mask = df['Expected Type'].apply(lambda v: len(v) > 1)
    df = df[mask]
    result = df[['Property', 'Expected Type']].set_index('Property').to_dict(orient='index')
    
    return {
        k: v['Expected Type']
        for k, v in result.items() 
    }


get_union_types(schemas['Dataset'])

{'audio': {'AudioObject', 'Clip'},
 'author': {'Organization', 'Person'},
 'contributor': {'Organization', 'Person'},
 'copyrightHolder': {'Organization', 'Person'},
 'creator': {'Organization', 'Person'},
 'funder': {'Organization', 'Person'},
 'isBasedOn': {'CreativeWork', 'Product'},
 'producer': {'Organization', 'Person'},
 'provider': {'Organization', 'Person'},
 'publisher': {'Organization', 'Person'},
 'sdPublisher': {'Organization', 'Person'},
 'sponsor': {'Organization', 'Person'},
 'translator': {'Organization', 'Person'},
 'video': {'Clip', 'VideoObject'},
 'subjectOf': {'CreativeWork', 'Event'}}

In [15]:
def get_schemas_union_types(schemas):
    union_types = {}

    for schema_name, schema_df in schemas.items():
        union_types.update(get_union_types(schema_df))
    return union_types


get_schemas_union_types(schemas)

{'audio': {'AudioObject', 'Clip'},
 'author': {'Organization', 'Person'},
 'contributor': {'Organization', 'Person'},
 'copyrightHolder': {'Organization', 'Person'},
 'creator': {'Organization', 'Person'},
 'funder': {'Organization', 'Person'},
 'isBasedOn': {'CreativeWork', 'Product'},
 'producer': {'Organization', 'Person'},
 'provider': {'Organization', 'Person'},
 'publisher': {'Organization', 'Person'},
 'sdPublisher': {'Organization', 'Person'},
 'sponsor': {'Organization', 'Person'},
 'translator': {'Organization', 'Person'},
 'video': {'Clip', 'VideoObject'},
 'subjectOf': {'CreativeWork', 'Event'},
 'attendee': {'Organization', 'Person'},
 'composer': {'Organization', 'Person'},
 'location': {'Place', 'PostalAddress'},
 'organizer': {'Organization', 'Person'},
 'performer': {'Organization', 'Person'},
 'brand': {'Brand', 'Organization'},
 'category': {'PhysicalActivityCategory', 'Thing'},
 'depth': {'Distance', 'QuantitativeValue'},
 'height': {'Distance', 'QuantitativeValue'}

In [16]:
# variables: type_defs, union_types
template_type = Template('''
const {
  GraphQLObjectType,
  GraphQLList,
  GraphQLString,
  GraphQLBoolean,
  GraphQLInt,
  GraphQLFloat,
  GraphQLUnionType
} = require('graphql');

{% for typename, fields in type_defs %}
const {{ typename }} = new GraphQLObjectType({
  name: '{{ typename }}',
  fields: () => ({
{% for field_name, field_types, field_description in fields %}
    {% 
      if field_name == 'identifier' 
    %}{{ field_name }}: { type: {{ field_types|safe }} },{%
      else
    %}{{ field_name }}: { type: GraphQLList({{ field_types|safe }}) },{% endif %}{#  // {{ field_description }}#}
{% endfor %}
  })
});
{% endfor %}

{% for typename, join_types in union_types %}
const {{ typename }} = new GraphQLUnionType({
  name: '{{ typename }}',
  types: [].concat(
    {% for t in join_types %}{{ t }},
    {% endfor %}
  ),
  resolveType(value) {
    return value.__typename;
  }
});
{% endfor %}

module.exports = {
  {% for typename, fields in type_defs %}{{ typename }},
  {% endfor %}
};

''')

# variables: input_types: [(input_type: str, deps:(str,))]
template_union_input = Template('''
// see: https://github.com/Cardinal90/graphql-union-input-type/issues/9

const {
  GraphQLObjectType,
  GraphQLList,
  GraphQLString,
  GraphQLBoolean,
  GraphQLInt,
  GraphQLFloat,
  GraphQLInputObjectType
} = require('graphql');

const { makeExecutableSchema } = require('apollo-server');
const UnionInputType = require('graphql-union-input-type');
let schema;

/* input types definition */
{% for input_type, fields in type_defs %}
const {{ input_type }}Input = new GraphQLInputObjectType({
  name: '{{ input_type }}Input',
  fields: () => ({
{% for field_name, field_types, field_description in fields %} {% 
  if field_name == 'identifier' 
%}   {{ field_name }}: { type: {{ field_types|safe }} },{%
  else
%}   {{ field_name }}: { type: GraphQLList({{ field_types|safe }}) },{% endif %}{#  // {{ field_description }}#}
{% endfor %}
  })
});
{% endfor %}
/* union type for input types definition */
{% for input_type, input_deps in input_types %}
const {{ input_type }}Input = UnionInputType({
  name: '{{ input_type }}Input',
  inputTypes: [
    {% for dep in input_deps %}{{ dep }}Input,
    {% endfor %}
  ],
});
{% endfor %}
/* AnyInput definition */
const AnyInput = UnionInputType({
  name: 'AnyInput',
  inputTypes: [
    {% for input_type, fields in type_defs %}{{ input_type }}Input, 
    {% endfor %}
  ],
  resolveType(name) {
    // The important part: return the type from the schema from its name (e.g. "FirstInputType")
    // console.log(schema.getTypeMap());
    return schema.getTypeMap()[name + 'Input'];
  },
  typeKey: '__typename'
});

/* resolvers */
resolvers = { AnyInput };

/* typeDefs */
const typeDefs = `
scalar AnyInput
{% for input_type, fields in type_defs %}scalar {{ input_type }}Input
{% endfor %}
`;

schema = makeExecutableSchema({ typeDefs, resolvers });

module.exports = { 
  AnyInput,
{% for typename, fields in type_defs %}  {{ typename }}Input,
{% endfor %}
};

''')

In [20]:
def get_schemas_type_defs(schemas):
    type_defs = {}
    
    for schema_name, schema_df in schemas.items():
        _df = schema_df.copy()

        mask_union = _df['Expected Type'].apply(lambda v: len(v) > 1)
        mask_single = _df['Expected Type'].apply(lambda v: len(v) == 1)

        _df['Expected Type'][mask_union] = _df[mask_union]['Property']
        _df['Expected Type'][mask_single] = _df[mask_single]['Expected Type'].apply(
            lambda v: list(v)[0]
        )
        typename = schema_df._metadata['typename']
        type_defs[typename] = _df[['Property', 'Expected Type', 'Description']].values
    return type_defs

get_schemas_type_defs(schemas)['Dataset']

array([['distribution', 'DataDownload',
        'A downloadable form of this dataset, at a specific location, in a specific format.'],
       ['includedInDataCatalog', 'DataCatalog',
        'A data catalog which contains this dataset. Supersedes catalog, includedDataCatalog. Inverse property: dataset.'],
       ['issn', 'GraphQLString',
        'The International Standard Serial Number (ISSN) that identifies this serial publication. You can repeat this property to identify different formats of, or the linking ISSN (ISSN-L) for, this serial publication.'],
       ['measurementTechnique', 'GraphQLString',
        'A technique or technology used in a Dataset (or DataDownload, DataCatalog),\ncorresponding to the method used for measuring the corresponding variable(s) (described using variableMeasured). This is oriented towards scientific and scholarly dataset publication but may have broader applicability; it is not intended as a full representation of measurement, but rather as a high le

In [18]:
def create_schema_type(schemas):
    type_defs = get_schemas_type_defs(schemas)
    union_types = get_schemas_union_types(schemas)

    result = template_type.render(
        type_defs=type_defs.items(),
        union_types=union_types.items()
    )

    return result.replace('\n\n', '\n')

result = create_schema_type(schemas)

print(result[:500] + '\n...\n' + result[-250:])


const {
  GraphQLObjectType,
  GraphQLList,
  GraphQLString,
  GraphQLBoolean,
  GraphQLInt,
  GraphQLFloat,
  GraphQLUnionType
} = require('graphql');

const Dataset = new GraphQLObjectType({
  name: 'Dataset',
  fields: () => ({
    distribution: { type: GraphQLList(DataDownload) },
    includedInDataCatalog: { type: GraphQLList(DataCatalog) },
    issn: { type: GraphQLList(GraphQLString) },
    measurementTechnique: { type: GraphQLList(GraphQLString) },
    variableMeasured: { type: GraphQLL
...
d,
  Clip,
  CreativeWork,
  Episode,
  CreativeWorkSeason,
  CableOrSatelliteService,
  DefinedTerm,
  RepaymentSpecification,
  ActionStatusType,
  Article,
  ServiceChannel,
  CreativeWorkSeries,
  DefinedTermSet,
  EntryPoint,
  Specialty,
  
};



In [19]:
def create_union_input_types(schemas):
    type_defs = get_schemas_type_defs(schemas)
    union_types = get_schemas_union_types(schemas)
    
    result = template_union_input.render(
        input_types=union_types.items(),
        type_defs=type_defs.items()
    )
    return result


print(create_union_input_types(schemas)[:1500] + ' ...')


// see: https://github.com/Cardinal90/graphql-union-input-type/issues/9

const {
  GraphQLObjectType,
  GraphQLList,
  GraphQLString,
  GraphQLBoolean,
  GraphQLInt,
  GraphQLFloat,
  GraphQLInputObjectType
} = require('graphql');

const { makeExecutableSchema } = require('apollo-server');
const UnionInputType = require('graphql-union-input-type');
let schema;

/* input types definition */

const DatasetInput = new GraphQLInputObjectType({
  name: 'DatasetInput',
  fields: () => ({
    distribution: { type: GraphQLList(DataDownload) },
    includedInDataCatalog: { type: GraphQLList(DataCatalog) },
    issn: { type: GraphQLList(GraphQLString) },
    measurementTechnique: { type: GraphQLList(GraphQLString) },
    variableMeasured: { type: GraphQLList(GraphQLString) },
    about: { type: GraphQLList(Thing) },
    accessMode: { type: GraphQLList(GraphQLString) },
    accessModeSufficient: { type: GraphQLList(GraphQLString) },
    accessibilityAPI: { type: GraphQLList(GraphQLString) },
   

In [17]:
path = '/home/xmn/dev/quansight/tmp/schemas'
# path = '/tmp/jupyterlab-metadata-service/backend/jupyterlab_metadata_service_server/src/schemas'
os.makedirs(path, exist_ok=True)

In [18]:
with open(os.path.join(path, 'schemaorg-typedef.js'), 'w') as f:
    f.write(create_schema_type(schemas))

In [19]:
with open(os.path.join(path, 'schemaorg-input.js'), 'w') as f:
    f.write(create_union_input_types(schemas))

## Extra

### GraphQL query and mutation example

**query**

```graphql
query {
    data: getByID(id: "schemaorg/Dataset/150") {
      ...on Dataset {
        identifier
        sameAs
        name
        creator {
          ...on Person {
            __typename
            identifier
          }
        }
      }
    }
}
```

**mutation**

```graphql
mutation {
  create(input: {
    __typename: "Dataset",
    name: ["dataset2"]
  }) {
    message
    success
    result {
      __typename
      ...on Dataset {
        identifier
        name
      }
    }
  }
}
```

### Fix initial json data

In [20]:
def fix_json_data():
    import json

    path = (
        '/tmp/jupyterlab-metadata-service/'
        'backend/jupyterlab_metadata_service_server/src/datasources/data/schemaorg.json-bkp'
    )

    with open(path) as f:
        data = json.load(f)

    for i, v in enumerate(data['Dataset']):
        data['Dataset'][i]['sameAs'] = data['Dataset'][i]['id']
        data['Dataset'][i]['identifier'] = 'schemaorg/Dataset/{}'.format(i)
        data['Dataset'][i]['__typename'] = 'Dataset'
        del data['Dataset'][i]['id']

        for c, w in v.items():
            if c in ('identifier', '__typename'):
                continue

            if isinstance(w, dict):
                w['identifier'] = 'schemaorg/{}'.format(w['id'].title())
                w['__typename'] = w['identifier'].split('/')[1]
                del w['id']

                for c2, w2 in w.items():
                    if c2 in ('identifier', '__typename'):
                        continue
                    w[c2] = [w2]
            
            data['Dataset'][i][c] = [w]

        data['Dataset'][i]['distribution'] = [{
            'url': v['distribution'][0]
        }]

    for k in ['Organization', 'Person']:
        for i, v in enumerate(data[k]):
            data[k][i]['identifier'] = 'schemaorg/{}'.format(data[k][i]['id'].title())
            data[k][i]['__typename'] = data[k][i]['identifier'].split('/')[1]
            del data[k][i]['id']

            for c, w in v.items():
                if c in ('identifier', '__typename'):
                    continue
                if isinstance(w, dict):
                    w['identifier'] = 'schemaorg/{}'.format(w['id'].title())
                    w['__typename'] = w['identifier'].split('/')[1]
                    del w['id']

                    for c2, w2 in w.items():
                        if c2 in ('identifier', '__typename'):
                            continue
                        w[c2] = [w2]
                data[k][i][c] = [w]

    path = (
        '/tmp/jupyterlab-metadata-service/'
        'backend/jupyterlab_metadata_service_server/src/datasources/data/schemaorg.json'
    )

    with open(path, 'w') as f:
        json.dump(data, f)
    print('[II] done!')

# fix_json_data()

### Example of Schema.org file content

1. Triples

```
<http://schema.org/URL> <http://www.w3.org/2000/01/rdf-schema#label> "URL" .
<http://schema.org/valueAddedTaxIncluded> <http://www.w3.org/2000/01/rdf-schema#comment> "Specifies whether the applicable value-added tax (VAT) is included in the price specification or not." .
<http://schema.org/paymentMethod> <http://schema.org/domainIncludes> <http://schema.org/Order> .
<http://schema.org/musicArrangement> <http://purl.org/dc/terms/source> <http://www.w3.org/wiki/WebSchemas/SchemaDotOrgSources#MBZ> .

...
```

2. Quads

```
<http://schema.org/CreativeWorkSeries> <http://www.w3.org/2000/01/rdf-schema#comment> "A CreativeWorkSeries in schema.org is a group of related items, typically but not necessarily of the same kind. CreativeWorkSeries are usually organized into some order, often chronological. Unlike <a class=\"localLink\" href=\"http://schema.org/ItemList\">ItemList</a> which is a general purpose data structure for lists of things, the emphasis with CreativeWorkSeries is on published materials (written e.g. books and periodicals, or media such as tv, radio and games).<br/><br/>\n\nSpecific subtypes are available for describing <a class=\"localLink\" href=\"http://schema.org/TVSeries\">TVSeries</a>, <a class=\"localLink\" href=\"http://schema.org/RadioSeries\">RadioSeries</a>, <a class=\"localLink\" href=\"http://schema.org/MovieSeries\">MovieSeries</a>, <a class=\"localLink\" href=\"http://schema.org/BookSeries\">BookSeries</a>
...
```

3. JSON-LD

```
{
  "@context": {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@graph": [
    {
      "@id": "http://schema.org/downloadUrl",
      "@type": "rdf:Property",
      "http://schema.org/domainIncludes": {
        "@id": "http://schema.org/SoftwareApplication"
      },
      "http://schema.org/rangeIncludes": {
        "@id": "http://schema.org/URL"
      },
...
```

4. RDF/XML

```
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:owl="http://www.w3.org/2002/07/owl#"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:dct="http://purl.org/dc/terms/"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:schema="http://schema.org/"
>
  <rdf:Property rdf:about="http://schema.org/valueAddedTaxIncluded">
    <rdfs:label>valueAddedTaxIncluded</rdfs:label>
    <dct:source rdf:resource="http://www.w3.org/wiki/WebSchemas/SchemaDotOrgSources#source_GoodRelationsTerms"/>
    <schema:domainIncludes rdf:resource="http://schema.org/PriceSpecification"/>
    <rdfs:comment>Specifies whether the applicable value-added tax (VAT) is included in the price specification or not.</rdfs:comment>
    <schema:rangeIncludes rdf:resource="http://schema.org/Boolean"/>
  </rdf:Property>

...
```

5. Turtle

```
@prefix schema: <http://schema.org/> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix dctype: <http://purl.org/dc/dcmitype/> .
@prefix eli: <http://data.europa.eu/eli/ontology#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfa: <http://www.w3.org/ns/rdfa#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix snomed: <http://purl.bioontology.org/ontology/SNOMEDCT/> .
@prefix void: <http://rdfs.org/ns/void#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix xsd1: <hhttp://www.w3.org/2001/XMLSchema#> .

schema:AMRadioChannel a rdfs:Class ;
    rdfs:label "AMRadioChannel" ;
    dct:source <https://github.com/schemaorg/schemaorg/issues/1004> ;
    schema:category "issue-1004" ;
    rdfs:comment "A radio channel that uses AM." ;
    rdfs:subClassOf schema:RadioChannel .
...
```


6. CSV

```
id;label;comment;subTypeOf;enumerationtype;equivalentClass;properties;subTypes;supersedes;supersededBy;isPartOf
http://schema.org/AMRadioChannel;AMRadioChannel;A radio channel that uses AM.;http://schema.org/RadioChannel;;;http://schema.org/additionalType, http://schema.org/alternateName, http://schema.org/broadcastChannelId, http://schema.org/broadcastFrequency, http://schema.org/broadcastServiceTier, http://schema.org/description, http://schema.org/disambiguatingDescription, http://schema.org/genre, http://schema.org/identifier, http://schema.org/image, http://schema.org/inBroadcastLineup, http://schema.org/mainEntityOfPage, http://schema.org/name, http://schema.org/potentialAction, http://schema.org/providesBroadcastService, http://schema.org/sameAs, http://schema.org/subjectOf, http://schema.org/url;;;;

...
```
